# Collect data from a forum (social media 1.0)

**Objectif** : Forum doctissimo (health) to extract urls used around the uses of cannabis & CBD

There is no API, we need to scrap the data

## 1. Collect data

### Test on a page

One URL

In [1]:
url = "https://forum.doctissimo.fr/sante/cannabis/liste_sujet-1.htm"

Get the page

In [2]:
from urllib.request import urlopen  # standard python module
html = urlopen(url).read()

The content

In [4]:
html[0:2000]

b'\n\t<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n\t<html xml:lang="fr" lang="fr">\n\t<head>\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n\t<meta name="viewport" content="" />\n<link rel="next" href="https://forum.doctissimo.fr/sante/cannabis/liste_sujet-2.htm" />\n<title>Forum Cannabis - Doctissimo</title><link rel="stylesheet" type="text/css" href="https://forum.doctissimo.fr/forum_style.php?is_responsive=0&amp;user=0&amp;id_forum=4&amp;color_key=FFFFFF/FFFFFF/333333/C2C3F4/42529d/FFFFFF/FFFFFF/333333/333333/333333/285291/FFFFFF/F8F8F8/F8F8F8/F8F8F8/C0C0C0/C0C0C0/FFFFFF/333333/333333/0000ff/FFEEEE/000000/FFFFFF/FF0000/FFFFFF/0/0/https%3A%40%40images.doctissimo.fr%40shared%40forum/NULL/&amp;v=1638174587" /><link type="text/css" rel="stylesheet" href="https://images.doctissimo.fr/generated/forum/compressed/pictures.css?v=1638174587" /><script type="text/javascript" src="https://

HTML : we need to look at the content, explore. We identify the html tage 'sujetCase3' with the content

To deal with HTML, we can use the `bs4` library

<td class="sujetCase3"> TOPIC </td>

In [10]:
import bs4

html_bs4 = bs4.BeautifulSoup(html)
data = html_bs4.find_all("td",{"class":"sujetCase3"})
len(data)

100

In [13]:
data[2]

<td class="sujetCase3" scope="row"><a class="cCatTopic" href="https://forum.doctissimo.fr/sante/cannabis/cannabis-douleurs-regles-sujet_162374_1.htm" id="url_topic_162374" title="Sujet n°162374">Cannabis et douleurs de règles</a></td>

In [9]:
type(html_bs4)

bs4.BeautifulSoup

What do we have ?

In [5]:
data[4]

<td class="sujetCase3" scope="row"><span class="DOC_cryptlinkaHR0cHM6Ly9mb3J1bS5kb2N0aXNzaW1vLmZyL3NhbnRlL2Nhbm5hYmlzL3Rlc3QtdXJpbmFpcmUtc3VqZXRfMTYyNDM5XzEuaHRt DOC_cryptlink cCatTopic" id="url_topic_162439" rel="nofollow" title="Sujet n°162439">Test urinaire thc</span></td>

In [33]:
data[6]

<td class="sujetCase3" scope="row"><a class="cCatTopic" href="https://forum.doctissimo.fr/sante/cannabis/reinsertion-socioprofessionnelle-consommateurs-sujet_162368_1.htm" id="url_topic_162368" title="Sujet n°162368">ECHANGES SUR LA RÉINSERTION SOCIOPROFESSIONNELLE D'EX-CONSOMMATEURS DE CANNABIS</a></td>

There is a link only if there are responses

We want also to get the titles

In [14]:
data[6].text

'Test urinaire thc'

In [9]:
textes = [i.text for i in data]

Get the links of the page

In [15]:
# Tableau vide
liens = []

#Boucle sur les liens
for ligne in data:
    
    # Chercher le lien
    lien = ligne.find("a")
    
    # S'il existe, le garder dans le tableau
    if lien:
        liens.append(lien["href"])
        
print(len(liens))

82


And now, we want the content of each of these pages of responses.

In [18]:
pages = {} # tableau vide
for lien in liens[0:10]: # on teste sur 10 pour commencer
    print(lien)
    html = urlopen(lien).read()
    pages[lien] = html

https://forum.doctissimo.fr/sante/cannabis/plateforme-numerique-addictions-sujet_162438_1.htm
https://forum.doctissimo.fr/sante/cannabis/cannabis-douleurs-regles-sujet_162374_1.htm
https://forum.doctissimo.fr/sante/cannabis/cannabis-menace-couple-sujet_162386_1.htm
https://forum.doctissimo.fr/sante/cannabis/test-positif-thc-sujet_162409_1.htm
https://forum.doctissimo.fr/sante/cannabis/guerire-addiction-cannabis-sujet_162437_1.htm
https://forum.doctissimo.fr/sante/cannabis/dure-arreter-sujet_162396_1.htm
https://forum.doctissimo.fr/sante/cannabis/reinsertion-socioprofessionnelle-consommateurs-sujet_162368_1.htm
https://forum.doctissimo.fr/sante/cannabis/grosse-addiction-habitude-sujet_162436_1.htm
https://forum.doctissimo.fr/sante/cannabis/test-urinaire-sujet_151489_1.htm
https://forum.doctissimo.fr/sante/cannabis/urine-consommation-soir-sujet_162235_1.htm


On each of thoses pages, we want to get the urls on the page.

In [19]:
def obtenir_liens(html):
    liens = bs4.BeautifulSoup(html).find_all("a",{"class":"cLink"})
    return [l["href"] for l in liens]

Let's test

In [20]:
obtenir_liens(pages[list(pages.keys())[2]])

['https://nox.tips/', 'https://xender.vip/']

Apply to all pages

In [21]:
liens_pages = {}
for p in pages:
    liens_pages[p] = obtenir_liens(pages[p])

Semi-structured data

In [22]:
liens_pages

{'https://forum.doctissimo.fr/sante/cannabis/plateforme-numerique-addictions-sujet_162438_1.htm': ['https://forms.gle/p3VVS28QRK1X9HSE9'],
 'https://forum.doctissimo.fr/sante/cannabis/cannabis-douleurs-regles-sujet_162374_1.htm': ['https://forms.gle/kpwj3uTsjaJvsvGi8'],
 'https://forum.doctissimo.fr/sante/cannabis/cannabis-menace-couple-sujet_162386_1.htm': ['https://nox.tips/',
  'https://xender.vip/'],
 'https://forum.doctissimo.fr/sante/cannabis/test-positif-thc-sujet_162409_1.htm': ['https://cocorikush.fr/blog/29_kleaner-spray-anti-thc.html',
  'https://cocorikush.fr/blog/29_kleaner-spray-anti-thc.html'],
 'https://forum.doctissimo.fr/sante/cannabis/guerire-addiction-cannabis-sujet_162437_1.htm': [],
 'https://forum.doctissimo.fr/sante/cannabis/dure-arreter-sujet_162396_1.htm': ['https://www.drogues-info-service.fr/Les-drogues-et-vous/L-arret/Arreter-comment-faire',
  'https://www.drogues-info-service.fr/Les-drogues-et-vous/L-arret/Est-il-possible-d-arreter-seul-le-cannabis',
  'ht

## 2. Explore and understand the data

We need to convert the partialy structured data to tables. We use `pandas`

In [23]:
import pandas as pd

In [24]:
pd.DataFrame([[p,len(liens_pages[p])] for p in liens_pages],columns=["page","nb liens"])

,page,nb liens
0,https://forum.doctissimo.fr/sante/cannabis/pla...,1
1,https://forum.doctissimo.fr/sante/cannabis/can...,1
2,https://forum.doctissimo.fr/sante/cannabis/can...,2
3,https://forum.doctissimo.fr/sante/cannabis/tes...,2
4,https://forum.doctissimo.fr/sante/cannabis/gue...,0
5,https://forum.doctissimo.fr/sante/cannabis/dur...,7
6,https://forum.doctissimo.fr/sante/cannabis/rei...,0
7,https://forum.doctissimo.fr/sante/cannabis/gro...,0
8,https://forum.doctissimo.fr/sante/cannabis/tes...,0
9,https://forum.doctissimo.fr/sante/cannabis/uri...,3


Are the links uniques ?

In [25]:
# un tableau vide
tous_les_liens = []

# ajouter chaque lien de chaque page
for p in liens_pages:
    tous_les_liens+=liens_pages[p]
    
# créer un tableau et compter
pd.Series(tous_les_liens).value_counts()

https://topcbd.ch/blog/                                                                                       3
https://cocorikush.fr/blog/29_kleaner-spray-anti-thc.html                                                     2
https://www.drogues-info-service.fr/Les-drogues-et-vous/L-arret/Arreter-comment-faire                         2
https://www.drogues-info-service.fr/Les-drogues-et-vous/L-arret/Est-il-possible-d-arreter-seul-le-cannabis    2
https://www.drogues-info-service.fr/Les-drogues-et-vous/L-arret/Je-souhaite-me-faire-aider                    2
https://forms.gle/p3VVS28QRK1X9HSE9                                                                           1
https://forms.gle/kpwj3uTsjaJvsvGi8                                                                           1
https://nox.tips/                                                                                             1
https://xender.vip/                                                                                     

## 3. Go to networks

We want  to build a network with the URL as a relation between threads

- Each link is a node
- Two nodes are connected if the link is on the same page

In [1]:
import networkx as nx

Build the graph

In [29]:
# Créer un graph vide
graph = nx.Graph()

# pour chaque page
for p in liens_pages:
    
    graph.add_node(p,poids=1,cat="page")
    
    # pour chaque lien de la page
    for l in liens_pages[p]:
        
        # ajouter le noeud ou augmenter sa taille
        if graph.has_node(l):
            graph.nodes[l]["poids"]+=1
        else:
            graph.add_node(l,poids=1,cat="lien")
            
        # ajouter le lien ou augmenter sa taille
        if graph.has_edge(p,l):
            graph[p][l]["poids"]+=1
        else:
            graph.add_edge(p,l,poids=1)        
        

The size of the network

In [23]:
len(graph.nodes),len(graph.edges)

(127, 109)

## 4. Visualisation


In [24]:
from ipysigma import Sigma

In [30]:
Sigma(graph,node_size="poids",node_color="cat")

Sigma(nx.Graph with 164 nodes and 109 edges)

## Questions pour le script

- comment faire pour récupérer plus de pages ?
- comment l'appliquer à un autre sujet ?
- comment faire pour savoir le nombre de sujets qui parlent de CBD ?